## Runs DE on different clusters. Also runs DE within each cluster for each condition.
Does this over each donor and combined

In [1]:
se_f = "/data/Mito_Trace/output/pipeline/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/SE.rds"
cells_meta_f = "/data/Mito_Trace/output/pipeline/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/mt_clones/thr__0.01_rt__10/cells_meta.tsv"
outdir = "/data/Mito_Trace/output/pipeline/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/thr__0.01_rt__10/annotation_clones/de_clone_btwnvars_RNA/minPct_0.1_logfc0_minCells1_pthresh0.1"
min_cells = 1 
sample_names= "Input,Flt3l"


# se_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/SE.rds"
# cells_meta_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/mt_clones/cells_meta.tsv"
# outdir = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/de_clone_btwnvars_RNA/"
# sample_names = "preB,postB"

#cells_meta_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/v2_MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/mt_clones/thr__0.01_rt__10/cells_meta.tsv"
#outdir = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/v2_MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/thr__0.01_rt__10/annotation_clones/de_clone_btwnvars_RNA/minPct_0_logfc0_minCells1_pthresh0.1_pthresh_0.1"
#min_cells = 1 

# Parameters
ncells_thresh = 10
minPct = 0.1
logfcthresh = 0
test.use="wilcox"
latent.vars = "NULL"
p_thresh = 0.1 
top_de = 3
ids.reversed = T

# Parallel processing
cores = 4
# R utils script
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"


In [2]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")
library(dplyr)
library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Attaching SeuratObject

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required pa

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [3]:
source(utils_script)

In [4]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
sample_names

[1] "Input" "Flt3l"

In [5]:
dir.create(outdir)

In [6]:
cells_meta <- read.table(cells_meta_f, sep="\t", header = T, row.names="cell")
cells_meta 

#cells_meta
ids.reversed = T

if (ids.reversed){
    print('here')
    split.names <- stringr::str_split(rownames(cells_meta), "_")
    rownames(cells_meta) = lapply(split.names, function(x){paste0(x[2], "_", x[1])})
}
cells_meta = cells_meta[, order(colSums(-cells_meta))]
head(cells_meta)

,X10302T,X10400T,X1040C,X10463C,X10516A,X10559G,X10571T,X10646A,X10734C,X1082G,⋯,X290T,X3514A,X3590A,X3599A,X3673C,X523C,X6431G,X9023T,X955T,X515G
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
AAACGAATCCCAGTAA-1_Input,0,0,1,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGAATCGTGAACT-1_Input,0,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACTCGAGGTACACG-1_Input,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACTCGCAAGACACG-1_Input,0,0,1,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACTGCAGGATGTAT-1_Input,0,0,0,0,1,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACTGCAGTCTCTAG-1_Input,0,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACTGCTCCTCCCAA-1_Input,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAAGATGCATGGGACA-1_Input,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAAGGATGTGATGTGG-1_Input,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


[1] "here"


,X11838A,X2514A,X7639T,X318A,X9173A,X14318C,X13263G,X7775A,X11719A,X2706G,⋯,X6961C,X6470C,X955T,X6904C,X3590A,X4855A,X9023T,X3003G,X3514A,X515G
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Input_AAACGAATCCCAGTAA-1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACGAATCGTGAACT-1,0,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACTCGAGGTACACG-1,0,0,0,0,0,0,1,1,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACTCGCAAGACACG-1,0,0,0,0,0,0,1,1,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACTGCAGGATGTAT-1,0,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACTGCAGTCTCTAG-1,0,0,0,0,0,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [7]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
se

An object of class Seurat 
189108 features across 5854 samples within 2 assays 
Active assay: RNA (19575 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

### Subset to just in cells_meta

In [8]:
se$inClones = 0
se[["inClones"]][(colnames(se) %in% row.names(cells_meta)), "inClones"] = 1

sum(se[["inClones"]])/dim(se[[]][1])

se.filt <- subset(se, subset = inClones == 1 )

[1]    0.7316365 4283.0000000

In [9]:
se.filt <- AddMetaData(se.filt, cells_meta)

In [10]:
se

An object of class Seurat 
189108 features across 5854 samples within 2 assays 
Active assay: RNA (19575 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [11]:
se.filt

An object of class Seurat 
189108 features across 4283 samples within 2 assays 
Active assay: RNA (19575 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [12]:
source(utils_script)
tmp <- get.clone.sizes(se.filt)
counts.conds.norm <- tmp[["counts.norm"]]
clone.sizes <- tmp[["clone.size"]]
head(clone.sizes)

`summarise()` has grouped output by 'condition', 'donor', 'lineage'. You can override using the `.groups` argument.

Warning message in xtfrm.data.frame(x):
"cannot xtfrm data frames"
`summarise()` has grouped output by 'donor'. You can override using the `.groups` argument.



donor,lineage,norm.total,total,name,cdf.norm,cdf,index
<fct>,<fct>,<dbl>,<int>,<fct>,<dbl>,<int>,<int>
0,10,0.24029451,40,0_10,0.1201473,40,1
0,13,0.19687842,33,0_13,0.2185865,73,2
0,23,0.11914324,19,0_23,0.2781581,92,3
0,12,0.11594864,43,0_12,0.3361324,135,4
0,2,0.11175003,66,0_2,0.3920074,201,5
0,11,0.09267373,47,0_11,0.4383443,248,6


In [13]:
dim(se.filt[[]])

[1] 4283  253

## plot each clone pre and post 

In [14]:
mt.vars <- colnames(cells_meta)
mt.vars

[1] "X11838A" "X2514A"  "X7639T"  "X318A"   "X9173A"  "X14318C" "X13263G"
  [8] "X7775A"  "X11719A" "X2706G"  "X8701G"  "X7196A"  "X14783C" "X12297C"
 [15] "X3552A"  "X7028T"  "X3010A"  "X4646C"  "X10873C" "X11914A" "X14766T"
 [22] "X4715G"  "X8642G"  "X7697A"  "X12308G" "X2291T"  "X8584A"  "X11467G"
 [29] "X15301A" "X5252A"  "X1040C"  "X3144G"  "X523C"   "X15043A" "X5999C" 
 [36] "X14620T" "X73G"    "X6047G"  "X11339C" "X10400T" "X10516A" "X11332T"
 [43] "X7705C"  "X3820A"  "X489C"   "X4837A"  "X12372A" "X8770T"  "X1949A" 
 [50] "X4117C"  "X3672G"  "X14905A" "X10559G" "X3630A"  "X5147A"  "X13108A"
 [57] "X15693C" "X12705T" "X14233G" "X1199T"  "X15607G" "X1888A"  "X11812G"
 [64] "X2083C"  "X2875G"  "X4216C"  "X9087A"  "X16519C" "X290T"   "X5580C" 
 [71] "X9545G"  "X9540C"  "X4917G"  "X7689A"  "X15184C" "X1811G"  "X499A"  
 [78] "X16362C" "X13368A" "X15452A" "X477C"   "X8118A"  "X16356C" "X11204C"
 [85] "X14861A" "X930A"   "X15789T" "X8352A"  "X10646A" "X4248C"  "X16126C"
 [92] "X6456A"  "X8697A"  "X16327T" "X16325C" "X5496G"  "X4820T"  "X16304C"
 [99] "X709A"   "X10398G" "X15122G" "X6043A"  "X5585T"  "X16051G" "X15487T"
[106] "X10463C" "X16294T" "X9899C"  "X16296T" "X15924G" "X11251G" "X16223T"
[113] "X16298C" "X13068G" "X3655A"  "X15928A" "X4806A"  "X1847A"  "X11382A"
[120] "X195C"   "X7849T"  "X8276A"  "X3296A"  "X150T"   "X10571T" "X13094A"
[127] "X9590T"  "X983T"   "X15235C" "X4100A"  "X62A"    "X15821A" "X16150T"
[134] "X8908A"  "X7579C"  "X16220C" "X12283T" "X12723C" "X3554A"  "X1551T" 
[141] "X302C"   "X10302T" "X9640T"  "X8609A"  "X249T"   "X14614T" "X13436A"
[148] "X513A"   "X6371A"  "X13619A" "X16512A" "X3735T"  "X5214A"  "X5337A" 
[155] "X4631T"  "X16387G" "X478C"   "X13305T" "X11137C" "X6207A"  "X8926T" 
[162] "X1192A"  "X6797T"  "X13443A" "X14962T" "X6364A"  "X2723T"  "X2694T" 
[169] "X4972A"  "X12944A" "X13339A" "X14991A" "X2653A"  "X13169C" "X5357C" 
[176] "X13850T" "X6142T"  "X6532C"  "X13246A" "X3402T"  "X7032A"  "X8285A" 
[183] "X2757G"  "X9476T"  "X5273T"  "X12079T" "X3673C"  "X8632C"  "X6891T" 
[190] "X5863G"  "X1471T"  "X1082G"  "X1334A"  "X9270A"  "X16227T" "X12790A"
[197] "X6431G"  "X6293C"  "X7218T"  "X769T"   "X7589A"  "X3599A"  "X10734C"
[204] "X6961C"  "X6470C"  "X955T"   "X6904C"  "X3590A"  "X4855A"  "X9023T" 
[211] "X3003G"  "X3514A"  "X515G"

In [15]:
se.filt[[]]

,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,⋯,X6961C,X6470C,X955T,X6904C,X3590A,X4855A,X9023T,X3003G,X3514A,X515G
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Input_AAACGAAAGGAAACTT-1,Input,2831,2831,49701,5002,164,92,1138,37812,5493,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACGAAGTCCAAGAG-1,Input,4565,4565,62869,9096,249,139,1902,42086,9397,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACGAAGTCCGAGCT-1,Input,7229,7229,60574,11583,352,141,2087,31767,14644,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACGAAGTTTGAAGA-1,Input,7001,7001,68829,12010,432,163,2209,39985,14030,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACGAATCCCAGTAA-1,Input,2103,2103,30056,2764,124,63,832,22385,3888,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACGAATCCGGGCAT-1,Input,3882,3882,41493,5928,189,120,951,26859,7446,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACGAATCCTGAAAC-1,Input,3683,3683,45177,5990,214,84,1332,31063,6494,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACGAATCGATGTGT-1,Input,6282,6282,51495,10502,267,133,2306,27006,11281,⋯,0,0,0,0,0,0,0,0,0,0
Input_AAACGAATCGCCAGAC-1,Input,2037,2037,22741,2223,77,28,556,16614,3243,⋯,0,0,0,0,0,0,0,0,0,0


In [16]:
for (mt in (mt.vars)){
    print(mt)    
    expr <- FetchData(se.filt, vars = mt)
    curr.se <- se.filt[, which(expr == 1)]
    #curr.se
    p <- DimPlot(se.filt , group.by = "condition", split.by=mt)
    p2 <- DimPlot(curr.se , split.by="condition")
    curr.count = sum(se.filt[[mt]])
    ggsave(file.path(outdir, paste("count", curr.count,mt, "groupCond.png", sep="_")), p)
    ggsave(file.path(outdir, paste("count", curr.count,mt, "clrCond.png", sep="_")), p2)
}

[1] "X11838A"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X2514A"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X7639T"


...

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X3514A"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X515G"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



In [17]:
outdir

[1] "/data/Mito_Trace/output/pipeline/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/thr__0.01_rt__10/annotation_clones/de_clone_btwnvars_RNA/minPct_0.1_logfc0_minCells1_pthresh0.1"

# Stimulus specific response in each clone

A. with variant vs without
B. top variants against top variants

In [18]:
#    expr <- FetchData(se.filt, vars = mt)
    #curr.se <- se.filt[, which(expr == 1)]
    #curr.se

In [20]:
all.pvals <- data.frame(matrix(nrow=dim(se.filt)[1],ncol=0),
                          row.names=row.names(se.filt))
all.names.sig <- c()

# #sample_names

for (curr.clone in mt.vars){
    print(curr.clone)
    clone.ids <- sort(unique(se.filt$name))
    se.filt$clone.stim <- paste(se.filt[[curr.clone]][,curr.clone], se.filt$condition,  sep = "_")

    # se$clone <- se$name
    Idents(se.filt) <- "clone.stim"

    ident.1 = paste0("0", "_", sample_names[[1]]) 
    ident.2 = paste0("1", "_", sample_names[[2]])
    try({
        print(curr.clone)
        curr.name <-paste0("clone_", curr.clone)
        curr.clone.outdir <- file.path(outdir, paste0("clone_", curr.clone))
        dir.create(curr.clone.outdir)

        curr.sig <- find.markers.and.plot(se.filt, ident.1, ident.2, curr.clone.outdir, 
                                          curr.name=curr.clone, assay="RNA", logfcthresh=logfcthresh,
                                          min.pct=minPct, p.thresh=p_thresh, mean.fxn=NULL, fc.name=NULL)
        names.sig <- rownames(curr.sig)
        all.names.sig <- c(all.names.sig, head(names.sig, top_de))
        all.pvals[names.sig,curr.clone] <- curr.sig$p_val_adj_BH        
    })
    }



all.pvals[rowSums(is.na(all.pvals)) != ncol(all.pvals), ]




[1] "X11838A"
[1] "X11838A"


Warning message in dir.create(curr.clone.outdir):
"'/data/Mito_Trace/output/pipeline/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/thr__0.01_rt__10/annotation_clones/de_clone_btwnvars_RNA/minPct_0.1_logfc0_minCells1_pthresh0.1/clone_X11838A' already exists"


[1] "se filt before"
[1] "se filt"
An object of class Seurat 
189108 features across 3050 samples within 2 assays 
Active assay: RNA (19575 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap
[1] "curr sig"
              p_val avg_log2FC pct.1 pct.2    p_val_adj p_val_adj_BH
GAS1   6.601409e-62 -0.2335816 0.064 0.492 1.292226e-57 1.029028e-57
KLF2   3.519219e-57 -0.2163964 0.063 0.469 6.888871e-53 2.742879e-53
ITM2C  7.661366e-55 -0.4387109 0.160 0.754 1.499712e-50 3.980846e-51
GP5    1.869723e-53 -0.1900627 0.046 0.385 3.659983e-49 7.261054e-50
WNT10A 2.329052e-53 -0.3020329 0.119 0.638 4.559120e-49 7.261054e-50
GATA3  1.598653e-50 -0.2623003 0.123 0.646 3.129363e-46 4.153301e-47
[1] "dim response"
[1] 15588     6


The automatically generated colors map from the 1^st and 99^th of the
values in the matrix. There are outliers in the matrix whose patterns
might be hidden by this color mapping. You can manually set the color
to `col` argument.

Use `suppressMessages()` to turn off this message.

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "gally"
[1] "p_val"        "avg_log2FC"   "pct.1"        "pct.2"        "p_val_adj"   
[6] "p_val_adj_BH"


...

The automatically generated colors map from the 1^st and 99^th of the
values in the matrix. There are outliers in the matrix whose patterns
might be hidden by this color mapping. You can manually set the color
to `col` argument.

Use `suppressMessages()` to turn off this message.

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "gally"
[1] "p_val"        "avg_log2FC"   "pct.1"        "pct.2"        "p_val_adj"   
[6] "p_val_adj_BH"


Saving 6.67 x 6.67 in image



[1] "X4855A"
[1] "X4855A"
[1] "se filt before"
Error in WhichCells.Seurat(object = x, cells = cells, idents = idents,  : 
  Cannot find the following identities in the object: 1_Flt3l
[1] "X9023T"
[1] "X9023T"
[1] "se filt before"
Error in WhichCells.Seurat(object = x, cells = cells, idents = idents,  : 
  Cannot find the following identities in the object: 1_Flt3l
[1] "X3003G"
[1] "X3003G"
[1] "se filt before"
Error in WhichCells.Seurat(object = x, cells = cells, idents = idents,  : 
  Cannot find the following identities in the object: 1_Flt3l
[1] "X3514A"
[1] "X3514A"
[1] "se filt before"
Error in WhichCells.Seurat(object = x, cells = cells, idents = idents,  : 
  Cannot find the following identities in the object: 1_Flt3l
[1] "X515G"
[1] "X515G"
[1] "se filt before"
Error in WhichCells.Seurat(object = x, cells = cells, idents = idents,  : 
  Cannot find the following identities in the object: 1_Flt3l


,X11838A,X2514A,X7639T,X318A,X9173A,X13263G,X7775A,X11719A,X8701G,X14783C,⋯,X1082G,X9270A,X6431G,X6293C,X7589A,X3599A,X10734C,X6961C,X955T,X3590A
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OR4F5,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
SAMD11,7.459215e-18,1.183172e-06,3.180157e-06,2.119380e-03,1.613614e-02,NA,4.290507e-04,0.0046975950,NA,NA,⋯,NA,NA,NA,NA,NA,9.964841e-06,NA,NA,NA,NA
NOC2L,4.939408e-05,1.958852e-02,1.079181e-03,NA,NA,NA,7.229426e-04,0.0052093875,NA,NA,⋯,0.0981921574,NA,NA,NA,NA,2.145988e-03,0.020582893,NA,0.041918014,NA
KLHL17,3.877439e-07,4.063965e-04,3.967252e-04,NA,3.077373e-03,0.02380200,2.641913e-04,0.0674724653,NA,0.083064395,⋯,0.0366952165,NA,NA,NA,NA,2.235105e-04,0.005018092,NA,0.012187157,NA
PLEKHN1,2.686818e-12,5.655843e-05,NA,7.173758e-05,4.542773e-02,NA,2.488228e-06,NA,NA,NA,⋯,0.0006126678,NA,7.475221e-08,5.700009e-05,NA,6.096829e-04,NA,NA,0.011642437,NA
PERM1,NA,NA,3.217910e-02,NA,NA,NA,NA,NA,NA,NA,⋯,0.0038576600,NA,NA,NA,0.0006319642,1.787507e-04,NA,NA,NA,2.420579e-02
HES4,4.065391e-07,4.538462e-02,3.781847e-07,NA,NA,NA,NA,NA,NA,0.006068038,⋯,NA,NA,NA,NA,NA,8.990018e-02,NA,NA,0.009787026,NA
ISG15,2.007922e-18,4.017967e-04,4.035935e-10,1.144949e-02,2.190622e-04,NA,2.606180e-02,NA,NA,NA,⋯,NA,NA,1.389492e-02,NA,NA,NA,NA,NA,NA,NA
AGRN,6.830466e-08,1.138430e-04,1.601573e-03,1.929599e-02,2.484567e-06,NA,2.434972e-03,0.0552159591,NA,NA,⋯,NA,NA,4.935194e-03,NA,NA,1.271906e-02,NA,NA,NA,NA


In [21]:
# ## DE sig summarize

# all.names.sig

# ### a) top 3 significant results from each comparison

# all.names.sig <- unique(all.names.sig)
# de.plots(se, all.names.sig, btwnConds_inClone_sepDonors.outdir, curr.name="allDonors.allClusters", max.size=15)

# dot <- DotPlot(se, scale=FALSE,
#                features = head(all.names.sig,20), 
#                cluster.idents=T) + RotatedAxis()
# dot
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.top3de.png"))
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.top3de.pdf"))



# ## b) averaged top p-values (only average ones that were DE)

# sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
# dot <- DotPlot(se, scale=T,
#                features = names(head(sig.all.ordered,20)), 
#                cluster.idents=T) + RotatedAxis()
# dot
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.pvalsOrdered.png"))
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.pvalsOrdered.pdf"))
# #de.plots(se, names(sig.all.ordered), outdir, curr.name="allDonors.allClusters.pvalsOrdered", max.size=20)

# sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
# dot <- DotPlot(se, scale=F,
#                features = head(names(sig.all.ordered),20), 
#                cluster.idents=T) + RotatedAxis()
# dot


In [22]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.3 LTS

Matrix products: default
BLAS/LAPACK: /home/isaac/miniconda3/envs/mttrace/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] future_1.23.0             dplyr_1.0.7              
 [3] RColorBrewer_1.1-2        metap_1.1                
 [5] cowplot_1.1.1             magrittr_2.0.1           
 [7] data.table_1.14.2         patchwork_1.1.1          
 [9] ggplot2_3.3.5             EnsDb.Hsapiens.v75_2.99.0
[11] 

In [23]:
outdir

[1] "/data/Mito_Trace/output/pipeline/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/thr__0.01_rt__10/annotation_clones/de_clone_btwnvars_RNA/minPct_0.1_logfc0_minCells1_pthresh0.1"